In [24]:
from dash import Dash, dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('product_segmentation.csv')

# App initialization
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Product Segmentation Dashboard", style={"textAlign": "center", "marginBottom": 20}),

    html.Div([
        html.Label("Select Segment:", style={"fontWeight": "bold"}),
        dcc.Dropdown(
            id='segment-dropdown',
            options=[{"label": seg, "value": seg} for seg in sorted(df['cluster_label'].unique())],
            value='Top Sellers',
            clearable=False,
            style={"width": "60%"}
        )
    ], style={"margin": "20px auto", "width": "80%"}),

    html.Div([
        html.Div([
            html.Div("📦", style={"fontSize": "30px", "textAlign": "center"}),
            html.H4("Avg Sales", style={"textAlign": "center", "margin": "5px 0"}),
            html.H5(id='avg-sales', style={"textAlign": "center", "fontWeight": "bold"})
        ], className="card"),

        html.Div([
            html.Div("💰", style={"fontSize": "30px", "textAlign": "center"}),
            html.H4("Avg Revenue", style={"textAlign": "center", "margin": "5px 0"}),
            html.H5(id='avg-revenue', style={"textAlign": "center", "fontWeight": "bold"})
        ], className="card"),

        html.Div([
            html.Div("⭐", style={"fontSize": "30px", "textAlign": "center"}),
            html.H4("Avg Review", style={"textAlign": "center", "margin": "5px 0"}),
            html.H5(id='avg-review', style={"textAlign": "center", "fontWeight": "bold"})
        ], className="card"),

        html.Div([
            html.Div("⏱️", style={"fontSize": "30px", "textAlign": "center"}),
            html.H4("Avg Delay", style={"textAlign": "center", "margin": "5px 0"}),
            html.H5(id='avg-delay', style={"textAlign": "center", "fontWeight": "bold"})
        ], className="card")
    ], style={
        "display": "flex",
        "justifyContent": "space-around",
        "marginTop": "20px",
        "flexWrap": "wrap"
    }),

    html.Div([
        html.H3("📌 Top 10 Products by Sales Volume", style={"marginBottom": 10}),
        dcc.Graph(id='bar-plot')
    ], style={"margin": "0 40px"}),

    html.Div([
        html.H3("📋 Product Table in Segment", style={"marginBottom": 10}),
        dash_table.DataTable(
            id='product-table',
            columns=[{"name": col, "id": col} for col in ['product_category_name', 'total_sales_volume', 'total_revenue', 'avg_review_score', 'avg_delivery_delay']],
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "center", "padding": "5px"},
            style_header={"fontWeight": "bold", "backgroundColor": "#f2f2f2"},
            page_size=10
        )
    ], style={"margin": "20px 40px"})

], style={"fontFamily": "Arial", "backgroundColor": "#fafafa"})

# Callback for interactivity
@app.callback(
    [Output('avg-sales', 'children'),
     Output('avg-revenue', 'children'),
     Output('avg-review', 'children'),
     Output('avg-delay', 'children'),
     Output('bar-plot', 'figure'),
     Output('product-table', 'data')],
    Input('segment-dropdown', 'value')
)
def update_dashboard(selected_segment):
    dff = df[df['cluster_label'] == selected_segment]

    # KPIs
    sales = dff['total_sales_volume'].mean().round(1)
    revenue = dff['total_revenue'].mean().round(1)
    review = dff['avg_review_score'].mean().round(2)
    delay = dff['avg_delivery_delay'].mean().round(2)

    # Format KPI values for display
    kpi_sales = f"{sales:,.0f}"
    kpi_revenue = f"{revenue:,.0f}"
    kpi_review = f"{review}"
    kpi_delay = f"{delay} days"

    # Bar plot
    top10 = dff.nlargest(10, 'total_sales_volume')
    fig = px.bar(
        top10,
        x='product_category_name',
        y='total_sales_volume',
        title='Top 10 Categories by Sales Volume',
        labels={'total_sales_volume': 'Sales Volume', 'product_category_name': 'Product Category'},
        template='plotly_white'
    )

    return kpi_sales, kpi_revenue, kpi_review, kpi_delay, fig, dff.to_dict('records')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
